In [4]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [5]:

# initialize the initial learning rate, number of epochs to train for,
# and batch size 
INIT_LR= 1e-4
EPOCHS = 5
BS = 32


# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
imagePaths = list(paths.list_images(r"C:\Users\SUMIT SINGH RAJPOOT\Downloads\acne-detection-model\acne-detection-master\dataset"))
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]

    # load the input image (224x224) and preprocess it
    image = load_img(imagePath,target_size=(224,224))
    print(image)
    image = img_to_array(image)
    image = preprocess_input(image)

    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)


# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels,
test_size=0.20, stratify=labels, random_state=42)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(
rotation_range=20,
zoom_range=0.15,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.15,
horizontal_flip=True,
fill_mode="nearest")

# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

# compile our model
print("[INFO] compiling model...")
INIT_LR = 1e-3
EPOCHS = 10
DECAY_STEPS = 1000
DECAY_RATE = 0.9

lr_schedule = ExponentialDecay(
    initial_learning_rate=INIT_LR,
    decay_steps=DECAY_STEPS,
    decay_rate=DECAY_RATE
)

opt = Adam(learning_rate=lr_schedule)
# model =(IMG_SIZE=IMG_SIZE, channels=channels, output_dim=len(characters), 
#                                          loss='binary_crossentropy', decay=1e-7, learning_rate=0.001, momentum=0.9,
#                                          nesterov=True)

model.compile(loss="binary_crossentropy", optimizer=opt,
metrics=["accuracy"])

# train the head of the network

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("facemodel", save_format="h5")

# plot the training loss and accuracy


[INFO] loading images...
<PIL.Image.Image image mode=RGB size=224x224 at 0x2245E553B50>
<PIL.Image.Image image mode=RGB size=224x224 at 0x22459D71430>
<PIL.Image.Image image mode=RGB size=224x224 at 0x22459D71430>
<PIL.Image.Image image mode=RGB size=224x224 at 0x22459D71430>
<PIL.Image.Image image mode=RGB size=224x224 at 0x22459D71430>
<PIL.Image.Image image mode=RGB size=224x224 at 0x2245E541F10>
<PIL.Image.Image image mode=RGB size=224x224 at 0x2245E585FD0>
<PIL.Image.Image image mode=RGB size=224x224 at 0x2245E553B50>
<PIL.Image.Image image mode=RGB size=224x224 at 0x2245E553B50>
<PIL.Image.Image image mode=RGB size=224x224 at 0x2245E553B50>
<PIL.Image.Image image mode=RGB size=224x224 at 0x2245E553B50>
<PIL.Image.Image image mode=RGB size=224x224 at 0x2245E553B50>
<PIL.Image.Image image mode=RGB size=224x224 at 0x2245E553B50>
<PIL.Image.Image image mode=RGB size=224x224 at 0x2245E553B50>
<PIL.Image.Image image mode=RGB size=224x224 at 0x2245E553B50>
<PIL.Image.Image image mode=RG

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [2]:
pip install imutils

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25872 sha256=9a5785852de1001592dabb660e7c35bad6c81337b417ee7936d4307c4546907c
  Stored in directory: c:\users\sumit singh rajpoot\appdata\local\pip\cache\wheels\4b\a5\2d\4a070a801d3a3d93f033d3ee9728f470f514826e89952df3ea
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.
